In [18]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import time

In [10]:
from gurobi_optimods import datasets
from gurobi_optimods.min_cost_flow import min_cost_flow_pandas


## Read data

In [267]:
# import the dataset
# Read the Excel file
file_path = 'input_parameters_real_mini.xlsx'

# Read the settings (time_limit, seed_number, nr, nl, m, block)
settings_df = pd.read_excel(file_path, sheet_name='Settings')
settings = settings_df.set_index('Parameter')['Value'].to_dict()

# time_limit = int(settings['time_limit'])
time_limit = 7200
seed_number = int(settings['seed_number'])
nr = int(settings['nr'])
nl = int(settings['nl'])
n = nr + nl
m = int(settings['m'])
block = int(settings['block'])

# Read the C_event matrix
C_event_df = pd.read_excel(file_path, sheet_name='C_event')
C_event = C_event_df.values
# take the (61,61) part of C_event
C_event = C_event[:m,:m]

# Read the C_home matrix
C_home_df = pd.read_excel(file_path, sheet_name='C_home')
C_home = C_home_df.values
C_home = C_home[:m]

# Read the C_dur array
C_dur_df = pd.read_excel(file_path, sheet_name='C_dur')
C_dur = C_dur_df['Duration'].values

# Read the time_window matrix
time_window_df = pd.read_excel(file_path, sheet_name='Time_Window')
time_window_flat = time_window_df.values
time_window = time_window_flat.reshape((m, block, 2))

# Read the minimum nurses required matrix
min_nurse_df = pd.read_excel(file_path, sheet_name='Min_Nurse')
min_nurse = min_nurse_df.values

In [268]:
# min_nurse_df.loc[7, 'RN']
min_nurse_df

,RN,LVN
0,2,0
1,2,0
2,2,0
3,1,0
4,2,0
5,3,0
6,1,0
7,1,0
8,2,0


In [269]:
# remove all rows with all zeros in time_window_df, reindex
time_window_df = time_window_df.loc[~(time_window_df == 0).all(axis=1)]
time_window_df = time_window_df.reset_index(drop=True)
time_window_df

,Start,End
0,1,1
1,0,1
2,1,1
3,1,1
4,1,1
5,1,1
6,1,0
7,0,1
8,1,0


In [270]:
def slot_type(event_index):
    if time_window_df.loc[event_index, 'Start'] == 1 and time_window_df.loc[event_index, 'End'] == 1:
        return "Full"
    elif time_window_df.loc[event_index, 'Start'] == 0 and time_window_df.loc[event_index, 'End'] == 1:
        return "PM"
    elif time_window_df.loc[event_index, 'Start'] == 1 and time_window_df.loc[event_index, 'End'] == 0:
        return "AM"
    else:
        return "None"


## Create network

In [271]:
# create node_data: a datafram with index and column name "demand"
# number of rows = 1 + n + 2*m + n + 1
node_data = pd.DataFrame(index=range(1 + n + 2*m + n + 1), columns=['demand'])

# fill in all demands with 0 for now
node_data['demand'] = 0
node_data

,demand
0,0
1,0
2,0
3,0
4,0
...,...
61,0
62,0
63,0
64,0


### edge data
Create edge data in steps: 

 - source -> homes(1); homes(2) -> sink
 - event(1) -> event(2)
 - home(1) -> event(1); event(2) -> home(2)
 - am event(2) -> pm event(1)
 - modify capacity and demand to account for minimum requirement

In [272]:
# create multiindex dataframe edge_data

edge_data = pd.DataFrame(index=pd.MultiIndex.from_product([range(1 + n + 2*m + n + 1), range(1 + n + 2*m + n + 1)]),
                        columns=['capacity', 'cost'])

# fill in all capacities with 0 for now
edge_data['capacity'] = 0
edge_data['cost'] = 0

# index (0, 1) to (0, n): set capacity = 1 
idx = pd.IndexSlice
edge_data.loc[idx[0, 1:n], 'capacity'] = 1
# index (1+n+2m, -1) to (2n+2m, -1): set capacity = 1
edge_data.loc[idx[1+n+2*m:2*n+2*m, 2*n + 2*m + 1], 'capacity'] = 1

We set "between-event" edges to have capacity n for now (because the package doesn't take lower bound).

In [273]:
# event nodes
# index (n+1, n+m+1), (n+2, n+m+2), ..., (n+m, n+2m): capacity = n
for i in range(m):
    edge_data.loc[idx[i + n + 1, i + n + m + 1], 'capacity'] = n

In [274]:
# first set of home nodes to first set of event nodes
for w in range(n):
    for i in range(m):
        # index (w+1, n+1), (w+1, n+2), ..., (w+1, n+m): capacity = 1, cost = C_home[i][w]
        edge_data.loc[idx[w + 1, i + n + 1], 'capacity'] = 1
        edge_data.loc[idx[w + 1, i + n + 1], 'cost'] = C_home[i][w]

        # index (n+m+1, n+2m+1+w), (n+m+2, n+2m+1+w), ..., (n+2m, n+2m+1+w): capacity = 1, cost = C_event[i][w]
        edge_data.loc[idx[i + n + m + 1, w + n + 2*m + 1], 'capacity'] = 1
        edge_data.loc[idx[i + n + m + 1, w + n + 2*m + 1], 'cost'] = C_home[i][w]

In [275]:
# am event to pm event
for i in range(m):
    for j in range(m):
        if slot_type(i) == "AM" and slot_type(j) == "PM":
            # index (i+n+1, j+n+m+1): capacity = 1, cost = C_event[i][j]
            edge_data.loc[idx[i + n + m + 1, j + n + 1], 'capacity'] = n
            edge_data.loc[idx[i + n + m + 1, j + n + 1], 'cost'] = C_event[i][j]

In [276]:
# connet sink to source
edge_data.loc[idx[2*n + 2*m + 1, 0], 'capacity'] = n
edge_data.loc[idx[2*n + 2*m + 1, 0], 'cost'] = 0

In [277]:
edge_data

capacity  cost
0  0          0     0
   1          1     0
   2          1     0
   3          1     0
   4          1     0
...         ...   ...
65 61         0     0
   62         0     0
   63         0     0
   64         0     0
   65         0     0

[4356 rows x 2 columns]

In [278]:
edge_data = edge_data.rename_axis(index=["source", "target"]) 
print(edge_data.head())

               capacity  cost
source target                
0      0              0     0
       1              1     0
       2              1     0
       3              1     0
       4              1     0


In [279]:
node_data = node_data.rename_axis(index="node")
node_data

,demand
node,
0,0
1,0
2,0
3,0
4,0
...,...
61,0
62,0
63,0


In [280]:
# save node and edge data
node_data.to_csv('node_data.csv')
edge_data.to_csv('edge_data.csv')

### Add minimum requirment
For every edge i -> i+m with lower bound min_req = min_nurse_df.loc[i, 'RN'], 

 - new capacity = capacity - min_req
 - demand at node i = demand + min_req
 - demand at node i+m = damand - min_req

In [281]:
for i in range(m):
    edge_data.loc[idx[i + n + 1, i + n + m + 1], 'capacity'] = n - min_nurse_df.loc[i, 'RN']
    node_data.loc[i + n + 1, 'demand'] += min_nurse_df.loc[i, 'RN']
    node_data.loc[i + n + m + 1, 'demand'] -= min_nurse_df.loc[i, 'RN']


In [228]:
# only keep the edges with capacity > 0
# edge_data = edge_data[edge_data['capacity'] > 0]

In [282]:
edge_data.tail()

capacity  cost
source target                
65     61             0     0
       62             0     0
       63             0     0
       64             0     0
       65             0     0

In [ ]:
# node_data.to_csv('node_data.csv', index=True)
# edge_data.to_csv('edge_data.csv', index=True)

In [283]:
# edge_data, node_data = datasets.simple_graph_pandas()
obj, sol = min_cost_flow_pandas(edge_data, node_data, verbose=True, solver_params={"Threads": 8, "Heuristics": 0})

Set parameter Username
Set parameter LicenseID to value 2595006
Set parameter Heuristics to value 0
Set parameter Threads to value 8
Academic license - for non-commercial use only - expires 2025-12-03
Solving min-cost flow with 66 nodes and 4356 edges
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M4 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 8 threads

Non-default parameters:
Heuristics  0
Threads  8

Optimize a model with 66 rows, 4356 columns and 8580 nonzeros
Model fingerprint: 0xa780f8cc
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 8e+01]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 3e+00]
Presolve removed 0 rows and 3882 columns
Presolve time: 0.00s
Presolved: 66 rows, 474 columns, 948 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.000000e+01   0.000000e+00      0s
      40    4.3400000

In [284]:
sol_df = pd.DataFrame(sol)
# sol_df = sol_df[sol_df['flow'] > 0]
sol_df

flow
source target      
0      0        0.0
       1        1.0
       2        1.0
       3        0.0
       4        1.0
...             ...
65     61       0.0
       62       0.0
       63       0.0
       64       0.0
       65       0.0

[4356 rows x 1 columns]

In [264]:
# sol_df.loc[idx[24, 33], 'flow']

### Recover the original flow

In [285]:
for i in range(m):
    sol_df.loc[idx[i + n + 1, i + n + m + 1], 'flow'] += min_nurse_df.loc[i, 'RN']

In [286]:
sol_df[sol_df['flow'] > 0].to_csv('solution.csv', index=True)

flow
source target      
0      1        1.0
       2        1.0
       4        1.0
       5        1.0
       6        1.0
...             ...
58     65       1.0
59     65       1.0
60     65       1.0
64     65       1.0
65     0       16.0

[65 rows x 1 columns]

In [287]:
with open('output.txt', 'w') as file:

    for index in sol_df.index:
        i, j = index  # Unpack the tuple
    
        if sol_df.loc[(i,j), 'flow'] > 0:
            if i == 0:
                file.write(f"Nurse {j}\n")

            elif j == 2*n + 2*m + 1:
                if i != 0:
                    file.write(f"Repeat: Nurse {i-n-2*m}\n")

            elif i in range(1, n+1):
                file.write(f"Nurse {i} to Event {j-n}\n")
            
            elif i in range(n+1, n+m+1):
                file.write(f"Event {i-n-1} scheduled (confirm Event {j-n-m-1}\n")
            
            elif i in range(n+m+1, n+2*m+1):
                file.write(f"Nurse {j-n-2*m} from Event {i-n-m}\n")

In [71]:
print(node_data.head())

   demand
0       0
1       0
2       0
3       0
4       0


In [16]:
node_data

,demand
0,-2
1,0
2,-1
3,1
4,0
5,2
